In [27]:
import pandas as pd
import numpy as np
from implicit.cpu.als import AlternatingLeastSquares
from scipy.sparse import csr_matrix


In [28]:
results = pd.read_csv("../benchmark/data/eval_results.csv")

results

,model_name,AUC_score@10,precision@10
0,fac100_reg0.05,0.582636,0.194799
1,fac100_reg0.01,0.582258,0.196217
2,fac200_reg0.05,0.543765,0.107171
3,fac300_reg0.01,0.525364,0.062411
4,fac300_reg0.05,0.525682,0.062727
5,fac300_reg0.1,0.526565,0.062727
6,fac200_reg0.01,0.543909,0.104807
7,fac50_reg0.01,0.600991,0.252167
8,fac50_reg0.05,0.600667,0.250591
9,fac200_reg0.1,0.543991,0.105910


In [29]:
max_AUC_ind = results["AUC_score@10"].argmax()

results.loc[max_AUC_ind]

model_name      fac50_reg0.01
AUC_score@10         0.600991
precision@10         0.252167
Name: 7, dtype: object

In [30]:
max_prec_ind = results["precision@10"].argmax()

results.loc[max_prec_ind]

model_name      fac50_reg0.01
AUC_score@10         0.600991
precision@10         0.252167
Name: 7, dtype: object

The ALS model with 50 latent factors and 0.01 regulariation parameter is the best

## Recommendation example

In [31]:
model = AlternatingLeastSquares.load("../models/fac50_reg0.01.npz")

In [32]:
interactions = pd.read_csv("../data/raw/u.data", sep="\t", header=None)
interactions.columns = ["user", "item", "rating", "timestamp"]

user_id = 126  # actual id (in raw dataset) is 127

interactions = interactions[(interactions["rating"] > 3) & (interactions["user"] == user_id + 1)]
liked_items = interactions["item"].values
liked_items  # actual ids (in raw dataset) are higher by one

array([229, 294, 380, 227, 300, 243, 748, 449, 288, 450, 222,  62, 271,
       343, 901, 228, 258, 230,  50])

In [33]:
recommendations, scores = model.recommend(user_id, csr_matrix(liked_items), N=1)
recommendation = recommendations[0]
recommendation

257

In [34]:
movies = pd.read_csv("../data/raw/u.item", sep="|", header=None, encoding="latin-1")

liked_movies = movies[1][liked_items + 1]
recommended_movie = movies[1][recommendation]

print("User liked movies:")
print(liked_movies.values)
print()
print("Recommended movie:")
print(recommended_movie)

User liked movies:
['Batman Returns (1992)' 'Promesse, La (1996)'
 'Adventures of Priscilla, Queen of the Desert, The (1994)'
 'Star Trek III: The Search for Spock (1984)' 'L.A. Confidential (1997)'
 "Devil's Own, The (1997)" 'Amistad (1997)' 'Grease (1978)'
 'Fierce Creatures (1997)' 'Jaws 2 (1978)' 'Ridicule (1996)'
 'Shawshank Redemption, The (1994)' 'Heat (1995)'
 'Deconstructing Harry (1997)' 'Afterglow (1997)'
 'Star Trek IV: The Voyage Home (1986)' 'Event Horizon (1997)'
 'Young Guns (1988)' 'Madness of King George, The (1994)']

Recommended movie:
Contact (1997)
